In [1]:
import tensorflow as tf
import pandas as pd
import yfinance as yf
import numpy as np
import math
from itertools import product, combinations
from sklearn.preprocessing import MinMaxScaler
from copy import copy
from collections import deque
from tqdm.auto import tqdm

In [2]:
class PortfolioEnv():
    def __init__(self, dates, datasets, n):
        self.dates = dates
        self.datasets = datasets
        self.n = n
        self.process_big_pt()
        
        self.process_small_pt()
        
        self.initial_pt = 1000000 # 1 000 000
        self.c_minus = 0.0025 # 0.25%
        self.c_plus = 0.0025 # 0.25%
        self.delta = 10000 # 10 000
                
        self.process_actions()
        self.action_shape = self.actions.shape[0]
        self._episode_ended = False
    
    def reset(self):
        # initialisation of portfolio
        self.pt = self.initial_pt # 1 000 000
        self.wt = [0.25,0.25,0.25,0.25] # wt before state 0
        
        self.current_tick = 0 # after checking, current_tick should be set to 0
        self.episode_ended = False
        
        ktc = self.big_pt[self.current_tick,0,:,self.n-1] # nouveaux close prices des différents assets
        wt_prime = (self.wt*self.phi(ktc)) / (np.dot(self.wt,self.phi(ktc))) # nouvelles proportions des assets du portfolio
        
        # after checking, we should return state 0
        return {'big_xt':np.array(self.small_pt[self.current_tick]), 'wt_prime':wt_prime}
    
    def process_actions(self):
        asset_number = 3
        action_number = 3

        seq = np.arange(asset_number)
        actions = []

        for c in product(seq, repeat=action_number):
            actions.append(c)

        self.actions = np.array(actions)
    
    def find_action_index(self,action):
        for ind, a in enumerate(self.actions):
            if np.array_equal(a, action):
                return ind;
    
    def process_big_pt(self):
        datasets = self.datasets
        date_start = self.dates[0]
        date_end = self.dates[1]

        dfs = []
        for d in datasets:
            ticker = yf.Ticker(d)
            # get historical market data
            df_ = ticker.history(start=date_start, end=date_end, interval="1d")    
            df_.rename(mapper={
                "Close": d+"_close",
                "Open": d+"_open",
                "High": d+"_high",
                "Low": d+"_low",
                "Volume": d+"_volume"
            }, inplace=True, axis=1)
            if "Dividends" in df_.columns:
                df_.drop(axis=1,labels=["Dividends", "Stock Splits"],inplace=True)
            dfs.append(df_)

        final_df = pd.concat(dfs, axis=1)
        final_df.dropna(inplace=True)
        
        self.final_df = final_df
        
        final_df = self.final_df
        n = self.n
        Pc = []
        for d in datasets:
            asset_close = final_df[d+"_close"].values
            asset_prev_close = final_df[d+"_close"].shift().values
            Kc = (asset_close - asset_prev_close) / asset_prev_close
            Kc = Kc[1:]
            Pc_ = [Kc[i:i+n] for i in range(len(asset_close)-n)] # Kc[0:20], Kc[1:21], Kc[2:22]
            Pc.append(Pc_)
        Pc = np.array(Pc)
        Po = []
        for d in datasets:
            asset_prev_close = final_df[d+"_close"].shift().values
            asset_open = final_df[d+"_open"].values
            Ko = (asset_open - asset_prev_close) / asset_prev_close
            Ko = Ko[1:]
            Po_ = [Ko[i:i+n] for i in range(len(asset_open)-n)]
            Po.append(Po_)
        Po = np.array(Po)
        Pl = []
        for d in datasets:
            asset_close = final_df[d+"_close"].values
            asset_low = final_df[d+"_low"].values
            Kl = (asset_close - asset_low) / asset_low
            Kl = Kl[1:]
            Pl_ = [Kl[i:i+n] for i in range(len(asset_low)-n)]
            Pl.append(Pl_)
        Pl = np.array(Pl)
        Ph = []
        for d in datasets:
            asset_close = final_df[d+"_close"].values
            asset_high = final_df[d+"_high"].values
            Kh = (asset_close - asset_high) / asset_high
            Kh = Kh[1:]
            Ph_ = [Kh[i:i+n] for i in range(len(asset_high)-n)]
            Ph.append(Ph_)
        Ph = np.array(Ph)
        Pv = []
        for d in datasets:
            asset_prev_volume = final_df[d+"_volume"].shift().values
            asset_volume = final_df[d+"_volume"].values
            Kv = (asset_volume - asset_prev_volume) / asset_prev_volume
            Kv = Kv[1:]
            Pv_ = [Kv[i:i+n] for i in range(len(asset_high)-n)]
            Pv.append(Pv_)
        Pv = np.array(Pv)
        Pt_star = np.array([Pc, Po, Pl, Ph, Pv])

        self.big_pt = Pt_star.swapaxes(0,2).swapaxes(1,2)
        print(self.big_pt.shape)
        
    def process_small_pt(self):
        small_pt = []
        
        for big_xt in self.big_pt:
            big_xt = big_xt.swapaxes(0,1).swapaxes(1,2)
            big_xt_scaled = np.array([mm_scaler.fit_transform(a) for a in big_xt])

            small_pt.append(big_xt_scaled)
        
        self.small_pt = np.array(small_pt)
        print(self.small_pt.shape)
    
    def phi(self,v):
        return np.insert(v, 0, [0]) + np.ones(len(v) + 1)
    
    def get_wt_prime_chapeau(self,wt_prime,big_s_minus,big_s_plus,pt_prime):
        wt_prime_chapeau = []
        for ind, key in enumerate(wt_prime):
            if(ind in big_s_minus):
                wt_prime_chapeau.append(key - self.delta/pt_prime)
            elif(ind in big_s_plus):
                wt_prime_chapeau.append(key + self.delta/pt_prime)
            else:
                wt_prime_chapeau.append(key)
    
        return np.array(wt_prime_chapeau)
    
    def is_asset_shortage(self,action,pt,wt):
        big_s_minus = np.where(action==0)[0]
        for ind in big_s_minus:
            if wt[ind+1]*pt < self.delta:
                return True
        
        return False
    
    #alright
    def is_cash_shortage(self,action,pt,wt):
        big_s_minus = np.where(action==0)[0]
        big_s_plus = np.where(action==2)[0]
        current_cash = wt[0]*pt
        cash_after_selling = current_cash + (1-self.c_minus)*self.delta*len(big_s_minus) # must include transaction costs
        cash_needed = (self.c_plus+1)*self.delta*len(big_s_plus) # must include transaction costs
        
        if(cash_after_selling < cash_needed):
            return True
        
        return False
    
    def action_mapping(self,action,action_Q_values,pt,wt):
        action = copy(action)
        action_mapped = action
        if self.is_asset_shortage(action,pt,wt):
            action_mapped = self.rule2(action,action_Q_values,pt,wt)
        elif self.is_cash_shortage(action,pt,wt):
            action_mapped = self.rule1(action,action_Q_values,pt,wt)
        
        return action_mapped
    
    def rule1(self,action,action_Q_values,pt,wt):
        MAXQ = np.NINF
        action_selected = action
        
        big_s_plus = np.where(action==2)[0]
        
        for i in range(1,len(big_s_plus) + 1):
            for c in combinations(big_s_plus, i):
                new_action = copy(action)
                for j in c:
                    new_action[j] = 1

                if not self.is_cash_shortage(new_action,pt,wt):
                    new_action_index = self.find_action_index(new_action)
                    new_action_Q_value = action_Q_values[new_action_index]
                    
                    if new_action_Q_value > MAXQ:
                        MAXQ = new_action_Q_value
                        action_selected = new_action
        
        return action_selected
    
    def rule2(self,action,action_Q_values,pt,wt):
        for i in range(len(action)):
            if wt[i+1]*pt < self.delta:
                action[i] = 1
        
        if self.is_cash_shortage(action,pt,wt):
            action = self.rule1(action,action_Q_values,pt,wt)
        
        return action
    
    def F(self,pt,wt):
        action_possible = []
        for ind, action in enumerate(self.actions):
            if not self.is_asset_shortage(action,pt,wt) and not self.is_cash_shortage(action,pt,wt):
                action_possible.append(ind)

        return np.array(action_possible)
    
    def step(self, action, simulation=False):
        # Must set new portfolio with regards to action
        # Must set new reward
        if self.current_tick == len(self.big_pt) - 2:
            # The last action ended the episode. Ignore the current action and start
            # a new episode.
            self.episode_ended = True
        
        # we are in state 0, best action between state 0 and state 1 has been predicted
        # so we must get portfolio value and weights after state 0 evolution
        # but before action has been taken into account
        ktc = self.big_pt[self.current_tick,0,:,self.n-1] # nouveaux close prices des différents assets
        pt_prime = self.pt * np.dot(self.wt,self.phi(ktc)) # nouvelle valeur du portfolio issue de l'action précédente
        wt_prime = (self.wt*self.phi(ktc)) / (np.dot(self.wt,self.phi(ktc))) # nouvelles proportions des assets du portfolio
        
        # On prend en compte la nouvelle action
        big_s_minus = np.where(action==0)[0]
        big_s_plus = np.where(action==2)[0]
        
        ct = (self.delta*(self.c_minus*len(big_s_minus) + self.c_plus*len(big_s_plus)))/pt_prime
        if not simulation:
            self.pt = pt_prime*(1 - ct)
        else:
            pt = pt_prime*(1 - ct)
        
        wt_prime_chapeau_1tillend = self.get_wt_prime_chapeau(wt_prime[1:],big_s_minus,big_s_plus,pt_prime)
        wt_prime_chapeau_0 = wt_prime[0] + self.delta*((1-self.c_minus)*len(big_s_minus)-(1+self.c_plus)*len(big_s_plus))/pt_prime
        wt_prime_chapeau = np.concatenate((np.array([wt_prime_chapeau_0]), wt_prime_chapeau_1tillend))
        
        # now we evolve to state one, to get reward of this action
        if not simulation:
            self.wt = wt_prime_chapeau / (np.dot(wt_prime_chapeau, np.ones(len(wt_prime_chapeau))))
            self.current_tick += 1
            k_t_plus_one_c = self.big_pt[self.current_tick,0,:,self.n-1]
        else:
            wt = wt_prime_chapeau / (np.dot(wt_prime_chapeau, np.ones(len(wt_prime_chapeau))))
            current_tick = self.current_tick + 1
            k_t_plus_one_c = self.big_pt[current_tick,0,:,self.n-1]
        
        big_p_s_t_plus_one = pt_prime*np.dot(wt_prime, self.phi(k_t_plus_one_c))
        
        if not simulation:
            p_t_plus_one_prime = self.pt * np.dot(self.wt,self.phi(k_t_plus_one_c))
            reward = (p_t_plus_one_prime - big_p_s_t_plus_one)/big_p_s_t_plus_one
            wt_plus_one_prime = (self.wt*self.phi(k_t_plus_one_c)) / (np.dot(self.wt,self.phi(k_t_plus_one_c)))
        else:
            p_t_plus_one_prime = pt * np.dot(wt,self.phi(k_t_plus_one_c))
            reward = (p_t_plus_one_prime - big_p_s_t_plus_one)/big_p_s_t_plus_one
            wt_plus_one_prime = (wt*self.phi(k_t_plus_one_c)) / (np.dot(wt,self.phi(k_t_plus_one_c)))
        
        if not simulation:
            return {'big_xt':np.array(self.small_pt[self.current_tick]), 'wt_prime':wt_plus_one_prime}, reward, self.episode_ended
        else:
            return {'big_xt':np.array(self.small_pt[current_tick]), 'wt_prime':wt_plus_one_prime}, reward, self.episode_ended, p_t_plus_one_prime, wt_plus_one_prime
        

In [3]:
mm_scaler = MinMaxScaler()

datasets = ["SPY", "IWD", "IWC"]
train_dates = ["2006-01-01", "2017-12-30"] # let's train it on more datas to see what happens
n = 20
env = PortfolioEnv(train_dates, datasets, n)
datas_ = env.big_pt
datas = datas_.swapaxes(2,3).swapaxes(1,3)
final_datas = []
for d in datas:
    final_datas.append(d[0])
    final_datas.append(d[1])
    final_datas.append(d[2])
final_datas = np.array(final_datas)
datas_scaled = np.array([mm_scaler.fit_transform(d) for d in final_datas]) # MinMaxScaler par ligne ou par colonne ?
X_train = datas_scaled[:7200]
X_valid = datas_scaled[7200:]

(3000, 5, 3, 20)
(3000, 3, 20, 5)


In [4]:
recurrent_encoder = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(units=128, input_shape=[20,5], return_sequences=True),
    tf.keras.layers.LSTM(units=20),
])
recurrent_decoder = tf.keras.models.Sequential([
    tf.keras.layers.RepeatVector(20, input_shape=[20]),
    tf.keras.layers.LSTM(units=20, return_sequences=True),
    tf.keras.layers.LSTM(units=128, return_sequences=True),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(5, activation="sigmoid"))
])
recurrent_ae = tf.keras.models.Sequential([recurrent_encoder, recurrent_decoder])
recurrent_ae.compile(optimizer='adam', loss='binary_crossentropy')
recurrent_encoder.summary()
recurrent_decoder.summary()
recurrent_ae.summary()

history = recurrent_ae.fit(X_train, X_train, epochs=50, validation_data=(X_valid, X_valid))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 128)           68608     
_________________________________________________________________
lstm_1 (LSTM)                (None, 20)                11920     
Total params: 80,528
Trainable params: 80,528
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
repeat_vector (RepeatVector) (None, 20, 20)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 20)            3280      
_________________________________________________________________
lstm_3 (LSTM)                (None, 20, 128)           76288     
______________________________

In [38]:
def create_envs(dates, datasets, n):
    return np.array([PortfolioEnv(d, datasets, n) for d in dates])
datasets = ["SPY", "IWD", "IWC"]
train_dates = [
    ["2010-01-01", "2010-12-30"],
    ["2011-01-01", "2011-12-30"],
    ["2012-01-01", "2012-12-30"],
    ["2013-01-01", "2013-12-30"],
    ["2014-01-01", "2014-12-30"],
    ["2015-01-01", "2015-12-30"],
    ["2016-01-01", "2016-12-30"],
    ["2017-01-01", "2017-12-30"],
]
train_envs = create_envs(train_dates, datasets, n)

(230, 5, 3, 20)
(230, 3, 20, 5)
(231, 5, 3, 20)
(231, 3, 20, 5)
(229, 5, 3, 20)
(229, 3, 20, 5)
(230, 5, 3, 20)
(230, 3, 20, 5)
(230, 5, 3, 20)
(230, 3, 20, 5)
(230, 5, 3, 20)
(230, 3, 20, 5)
(231, 5, 3, 20)
(231, 3, 20, 5)


In [39]:
test_dates = [
    ["2018-01-01", "2018-12-30"],
]
test_envs = create_envs(test_dates, datasets, n)

(231, 5, 3, 20)
(231, 3, 20, 5)


In [58]:
import time
import pickle as pkl

@tf.function
def train_step(preprocessed_states, wts_prime, mask, target_Q_values):
    with tf.device('gpu:0'):
        with tf.GradientTape() as tape:
            all_Q_values = model((wts_prime, preprocessed_states[:,0], preprocessed_states[:,1], preprocessed_states[:,2]))
            Q_values = tf.reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
            loss = tf.reduce_mean(loss_fn(target_Q_values, Q_values))
            # Q_values ne concerne pas les actions pas concernées, de même que target_Q_values
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

beta = .3
epochs = 50 

r = np.random.rand(epochs)
N=len(train_envs)
gen_trunc=(N-1-np.floor(np.log(1-r*(1-(1-beta)**N))/np.log(1-beta))).astype(int)

tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

K = tf.keras.backend

wt_prime_inpt = tf.keras.layers.Input(shape=[4])
inputs = [wt_prime_inpt]
encoders = []
for t in datasets:
    inpt = tf.keras.layers.Input(shape=[20, 5])
    inputs.append(inpt)
    encoder = recurrent_encoder(inpt)
    encoders.append(encoder)

enc_concat = tf.keras.layers.Concatenate(axis=1)(encoders)
wt_prime_concat_with_enc = tf.keras.layers.Concatenate(axis=1)([wt_prime_inpt, enc_concat])

hidden1 = tf.keras.layers.Dense(64, activation="relu")(wt_prime_concat_with_enc)
hidden2 = tf.keras.layers.Dense(32, activation="relu")(hidden1)
output = tf.keras.layers.Dense(train_envs[0].action_shape)(hidden2)

model = tf.keras.models.Model(inputs=[inputs], outputs=[output])
model.summary()

target = tf.keras.models.clone_model(model)
target.set_weights(model.get_weights())

replay_memory = deque(maxlen=2000)

epsilon = 1
batch_size = 32
discount_rate = .9

optimizer = tf.keras.optimizers.Adam(lr=1e-7)
loss_fn = tf.keras.losses.mean_squared_error

steps = 0
episode_counter = 1

train_CR = [[] for env in train_envs]
previous_CR = 0

test_CR = []
test_previous_CR = 0

for ind_env in gen_trunc:
    env = train_envs[ind_env]
    state = env.reset()
        
    pbar = tqdm(total=len(env.big_pt)-1)
        
    while 1: # will break when end of env
        pbar.set_description(f'e.{episode_counter}/{len(gen_trunc)}|tr.p.CR:{previous_CR}|te.p.CR:{test_previous_CR}')
        #epsilon-greedy policy to select an action
        action = None        

        k_t_plus_one_c = env.big_pt[env.current_tick,0,:,env.n-1]
        p_t_plus_one_prime = env.pt * np.dot(env.wt,env.phi(k_t_plus_one_c))
        wt_plus_one_prime = (env.wt*env.phi(k_t_plus_one_c)) / (np.dot(env.wt,env.phi(k_t_plus_one_c)))
        
        if np.random.rand() < epsilon: # goal is to take uniformly random action among possible actions of this env
            possible_actions = env.F(p_t_plus_one_prime, wt_plus_one_prime) # when we sell/buy it's new prices, not protfolio
            # value when action A has been taken, so when action A has been taken and then new state has come

            random_index = np.random.randint(len(possible_actions))
            action = env.actions[possible_actions[random_index]]
        else: # here goal is to make model predict best action, but if action isn't feasable for env, map it
            Q_values = model.predict((state['wt_prime'][np.newaxis], state['big_xt'][0][np.newaxis], state['big_xt'][1][np.newaxis], state['big_xt'][2][np.newaxis]))
            action = env.action_mapping(env.actions[np.argmax(Q_values[0])], Q_values[0], p_t_plus_one_prime, wt_plus_one_prime)
        
        # simulate all feasible actions for current state
        possible_actions = env.F(p_t_plus_one_prime, wt_plus_one_prime)
        simulations = []
        
        for simulated_action in possible_actions:
            next_state_simulated, next_reward, done, next_pt, next_wt = env.step(env.actions[simulated_action], simulation=True)

            simulations.append((state['big_xt'], state['wt_prime'], simulated_action, next_reward, next_state_simulated['big_xt'], done, next_pt, np.array(next_wt)))
        replay_memory.append(np.array(simulations))
        next_state, reward, episode_ended = env.step(action)
        # do some stuff with it
        
        # add some time for buffer to populate
        
        if steps >= 100:
            # start batch training (room to improvment, why don't use Prioritized Experience Replay ?)
            # select a uniformly random batch
            indices = np.random.randint(len(replay_memory), size=batch_size)
            batch = [replay_memory[index] for index in indices]
            for simulations in batch:
                #start = time.time()
                preprocessed_states, wts_prime, actions, rewards, preprocessed_next_states, dones, next_pts, next_wts = [np.array([simulation[field_index] for simulation in simulations]) for field_index in range(8)]
                
                next_Q_values = target.predict((next_wts, preprocessed_next_states[:,0], preprocessed_next_states[:,1], preprocessed_next_states[:,2])) # Q-values predicted for all next_states
                max_next_Q_values_index = np.argmax(next_Q_values, axis=1) # for each next_states find index of max Q-value predicted by target
                
                # okay, here, in max_next_Q_values, there are some actions that may not be feasible and that we need to map
                # but we have to use is_asset_shortage, is_cash_shortage and action_mapping with state of simulation
                max_next_Q_values_actions = np.array([
                    env.actions[action_simulated] if not env.is_asset_shortage(env.actions[action_simulated], next_pts[ind], next_wts[ind]) and not env.is_cash_shortage(env.actions[action_simulated], next_pts[ind], next_wts[ind])
                    else env.action_mapping(env.actions[action_simulated], next_Q_values[ind], next_pts[ind], next_wts[ind])
                    for ind, action_simulated in enumerate(max_next_Q_values_index)
                ])
                
                # okay, we have the feasible actions selected by the target network, now we need to find their indexes to communicate with the network
                max_next_Q_values_indexes = [env.find_action_index(action_simulated) for action_simulated in max_next_Q_values_actions]
                # now we have to find the Q-values predicted by target network of these corrected actions
                max_next_Q_values = [next_Q_values[ind][best_action_simulated] for ind, best_action_simulated in enumerate(max_next_Q_values_indexes)]

                # and now, we can proprely calculate the target Q-value
                target_Q_values = (rewards +
                                   (1 - dones) * discount_rate * max_next_Q_values) # here problem with dones /!\
                target_Q_values = target_Q_values.reshape(-1, 1)
                mask = tf.one_hot(actions, env.action_shape)
                
                train_step(preprocessed_states, wts_prime, mask, target_Q_values)
                # and we're done, amazing !!
                #print('updates ::: {:.5f} ms / step'.format((time.time() - start) * 1000))
            
        state = next_state

        steps += 1
        pbar.update(1)
        
        if episode_ended:
            break;
        
    episode_counter += 1
    
    previous_CR = round((env.pt-env.initial_pt)/env.initial_pt, 3)
    train_CR[ind_env].append(previous_CR)
    pbar.close()
    
    for test_env in test_envs:
        test_state = test_env.reset()
        # state 0 return
        # but current_tick set to one
        test_episode_ended = False
        while not test_episode_ended:
            # state 0
            # Q-values of action from state 0 predicted
            # algorithm only knows state 0 and want to predict best action and then state one happens
            
            test_Q_values = model.predict((test_state['wt_prime'][np.newaxis], test_state['big_xt'][0][np.newaxis], test_state['big_xt'][1][np.newaxis], test_state['big_xt'][2][np.newaxis]))
            
            # we want to get total value of portfolio and new portfolio weights regarding state 0 evolution
            # so we enter the algorithm with an already predefined portfolio distribution
            # then state 0 happens
            # so here current_tick should 0, not one,
            # to calculate total value of portfolio and new portfolio weights regarding state 0 evolution
            test_k_t_plus_one_c = test_env.big_pt[test_env.current_tick,0,:,test_env.n-1]
            test_p_t_plus_one_prime = test_env.pt * np.dot(test_env.wt,test_env.phi(test_k_t_plus_one_c))
            test_wt_plus_one_prime = (test_env.wt*test_env.phi(test_k_t_plus_one_c)) / (np.dot(test_env.wt,test_env.phi(test_k_t_plus_one_c)))
            
            test_action = test_env.action_mapping(test_env.actions[np.argmax(test_Q_values[0])], test_Q_values[0], test_p_t_plus_one_prime, test_wt_plus_one_prime)
            
            # okay here, we have selected best action predicted after state 0 evolution
            # and we want to evolve to state one
            test_state, test_reward, test_episode_ended = test_env.step(test_action)
        
        test_previous_CR = round((test_env.pt-test_env.initial_pt)/test_env.initial_pt, 3)
        test_CR.append(test_previous_CR)
        
    # update of target network
    target.set_weights(model.get_weights()) 
    epsilon = (1-episode_counter/500) # max(1 - episode_counter / 400, 0.01)
    
    if episode_counter % 25 == 0:
        checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
        model.save_weights(checkpoint_path.format(epoch=episode_counter))
        pkl.dump(train_CR, open('train_CR.pkl', 'wb'))
        pkl.dump(test_CR, open('test_CR.pkl', 'wb'))
        
# In[ ]:
pkl.dump(train_CR, open('train_CR.pkl', 'wb'))
pkl.dump(test_CR, open('test_CR.pkl', 'wb'))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 20, 5)]      0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 20, 5)]      0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 20, 5)]      0                                            
__________________________________________________________________________________________________
sequential_3 (Sequential)       (None, 20)           80528       input_2[0][0]                    
                                                                 input_3[0][0]                


1000000
1000000



1000000
1000000



1000000
1000000



1000000
1000000


...


1000000
1000000



1000000
1000000



1000000
1000000



1000000
1000000


KeyboardInterrupt: 